In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

**CODING QUESTIONS TRAINING**

In [2]:
#1) Resize the training images to 128 x 128.

In [3]:
#2) Computer HoG features of size (32,32,8). Apply blocknorm in 4x4 cell neighborhood. (Deduce values of other required variables)

In [4]:
#3) Fit a nearest neighbor classifier with three neighbors. Use KNeigborsClassifier from sklearn library.


**CODING QUESTIONS TESTING**

In [5]:
#1) Resize test images to 128 x 128.

In [6]:
#2) Compute HoG features similar to what was done during training.

In [7]:
#3) Display the features for both test images.

In [8]:
#4) Classify the test images using the classifier built above.

**REASONING QUESTIONS**

_1. Will HoG work if images of logos undergo random rotation? If yes, which proprety of HoG avails this feature? If not, discuss the sequence of computer vision technique(s) that can be used in order to regain uniform orientation of the logos._

_2. Consider that you are given a large dataset with three types of images: variable sized squares with either or both diagonals drawn. How many and which directions of the gradient are sufficient in order to correctly classify the images into three categories?_